In [1]:
import os
import sys
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import com_func

def dummy(doc):
    return doc

# parameters
threshold = 30
cutoff = 3

pp_textual = "tf"

Dataset = "pubmed"

In [2]:
import pickle
import gensim
# read trained rec to rec textual graph
def read_textual_embedding(Dataset = "pubmed", emb_type = "off"):
    textual_emb = []
    emb_pid = []
    while True:
        if emb_type == "tf":
            modelSaveDir = "../Data/"+Dataset+"/models/tf/textual_sample=140k/"
            with open(modelSaveDir+'tf_features.pickle', "rb") as input_file:
                vec = pickle.load(input_file)
            with open(modelSaveDir+'feature_pid.pickle', "rb") as input_file:
                allPaperid = pickle.load(input_file)
            textual_emb = vec.toarray()
            emb_pid = allPaperid
            break
        elif emb_type == "tfidf":
            modelSaveDir = "../Data/"+Dataset+"/models/tf_idf/textual_sample=140k/"
            with open(modelSaveDir+'tf_idf_trained_features.pickle', "rb") as input_file:
                vec = pickle.load(input_file)
            with open(modelSaveDir+'feature_pid.pickle', "rb") as input_file:
                allPaperid = pickle.load(input_file)
            textual_emb = vec.toarray()
            emb_pid = allPaperid
            break
        elif emb_type == "lsa":
            modelSaveDir = "../Data/"+Dataset+"/models/lsa/textual_sample=140k/"
            with open(modelSaveDir+'lsa_Matrix.pickle', "rb") as input_file:
                vec = pickle.load(input_file)
            with open(modelSaveDir+'feature_pid.pickle', "rb") as input_file:
                allPaperid = pickle.load(input_file)
            textual_emb = vec
            emb_pid = allPaperid
            break
        elif emb_type == "pv_dm":
            modelSaveDir = "../Data/"+Dataset+"/models/doc2v/textual_sample=140k/"
            model = gensim.models.Doc2Vec.load(modelSaveDir+"pv_dm/Doc2Vec(dmm,d100,n5,w5,mc2,s0.001,t24)")
            allPaperTags = model.docvecs.offset2doctag
            for pid in allPaperTags:
                vectorRepresentation = model.docvecs[pid].tolist()
                vectorRepresentation = [float(i) for i in vectorRepresentation]
                textual_emb.append(vectorRepresentation)
            emb_pid = allPaperTags
            break
        elif emb_type == "pv_dbow":
            modelSaveDir = "../Data/"+Dataset+"/models/doc2v/textual_sample=140k/"
            model = gensim.models.Doc2Vec.load(modelSaveDir+"pv_dbow/Doc2Vec(dbow,d100,n5,mc2,s0.001,t24)")

            allPaperTags = model.docvecs.offset2doctag
            for pid in allPaperTags:
                vectorRepresentation = model.docvecs[pid].tolist()
                vectorRepresentation = [float(i) for i in vectorRepresentation]
                textual_emb.append(vectorRepresentation)
            emb_pid = allPaperTags
            break
        elif emb_type == "off":
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="off"
    print("Total textual vector records:",len(textual_emb))
    print("Vector dimension: ", len(textual_emb[0]))
    return textual_emb, emb_pid

In [3]:
def extract_embedding(all_embedding, all_embedding_pid, wanted_pid_list):
    extracted_emb = []
    wanted_pid_list = wanted_pid_list.values.tolist()
    total_missing_sample = 0
    # only if embedding exist
    if len(all_embedding)>0:
        # loop through wanted pid list to keep input order
        for wanted_pid in wanted_pid_list:
            # if wanted paper in all pretrained embeddings
            if wanted_pid in all_embedding_pid:
                emb_idx = all_embedding_pid.index(wanted_pid)
                extracted_emb.append(all_embedding[emb_idx])
            # if wanted paper not in all pretrained embeddings, fill missing sample with 0's
            else:
                total_missing_sample+=1
                print("Missing Sample: ", wanted_pid)
                temp = [0] * len(all_embedding[0])
                extracted_emb.append(temp)
    print("Total missing sample: ", total_missing_sample)
    extracted_emb = pd.DataFrame(extracted_emb)
    return extracted_emb

In [4]:
def dummy(doc):
    return doc
def read_labeled_file(infile):
    LabeledRecords_original = []
    with open(infile, 'r', encoding = 'utf8') as f:
        for line in f:
            read_data = line.split("\t")
            # get ride of bad formated lines
            if(len(read_data)==13 or len(read_data)==12):
                paper_detail = {"paperID": read_data[0], "authorID":read_data[1], 
                                "co-author": read_data[5], "venue_id": read_data[7]}
                LabeledRecords_original.append(paper_detail)
            else:
                print(len(read_data))
        f.close()
    return pd.DataFrame(LabeledRecords_original)

In [5]:
# remove author(positive sample) from other(negative sample)
import random
def extractNegativeSample(positiveSample, allSample):
    negativeSample = [x for x in allSample if x not in positiveSample]
    return negativeSample

In [6]:
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score,f1_score,accuracy_score
# cross validation
def k_fold_cv(data, label, clf, k=10):
    kf = StratifiedKFold(n_splits=k, shuffle=False)
    allTrueLabel = []
    allPredLabel = []
    for train_index, test_index in kf.split(data, label):
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # split train and test
        data_train, data_test = data[train_index], data[test_index]
        label_train, label_test = label[train_index], label[test_index]
        # fit data to clf
        clf.fit(data_train, label_train)
        # get predicted label
        label_pred = clf.predict(data_test)
        allTrueLabel.extend(label_test)
        allPredLabel.extend(label_pred)
        # print("True positive: {tp}, False positive: {fp}, False negative: {fn}, True negative: {tn}"
        # .format(tp=round_tp, fp=round_fp, fn=round_fn, tn=round_tn))

    accuracy = accuracy_score(allTrueLabel, allPredLabel)
    f1 = f1_score(allTrueLabel, allPredLabel,average='macro')
    
    print(metrics.classification_report(allTrueLabel, allPredLabel))
    print(metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel())
    
    return accuracy, f1

In [11]:
from sklearn.base import clone
# self defined one vs rest
class one_vs_rest:
    
    def __init__ (self):
        self.binary_clf = []

    def fit_one_vs_rest(self, train_data, labeled_data, clf, clf_type):
        self.clf_type = clf_type
        self.classes = np.unique(labeled_data["authorID"]).tolist()
        # check for "RARE_VALUE" special marker that only used for train binary classifier, not creating new class
        for author in self.classes:
            if author =="RARE_VALUE":
                pass
            else:
                print("Binary clf: ", author)
                mask = labeled_data["authorID"] == author
                temp = labeled_data[mask]
                positive_sample_pid = temp["paperID"].tolist()
                negative_sample_pid = extractNegativeSample(positive_sample_pid, all_labeled_sample)
                # append to statistic collection
                positive_sample_size.append(len(positive_sample_pid))
                negative_sample_size.append(len(negative_sample_pid))
                # form positive and negative (negative class come from similar name group)
                all_authors = []
                all_authors.append(positive_sample_pid)
                all_authors.append(negative_sample_pid)
                appended_data = []
                for label, pid in enumerate(all_authors):
                    # create df save one author data 
                    authordf = pd.DataFrame({"paperID":pid})
                    authordf['label'] = label
                    appended_data.append(authordf)
                processed_data = pd.concat(appended_data, axis=0,ignore_index=True)
                
                # alignment 
                processed_data = pd.merge(labeled_data["paperID"].to_frame(), processed_data, on="paperID")
                
                # extract true label and it's corresponeding pid for check
                label = processed_data["label"]
                pid = processed_data["paperID"]
                

#                 LRaccuracy, LRmarcof1 = k_fold_cv(train_data, label, clf, k=10)
#                 print("LR Accuracy: ",LRaccuracy)
#                 print("LR F1: ", LRmarcof1)

                # using converted feature vector to train classifier
                traing_clf = clone(clf)
                traing_clf.fit(train_data, label)
                print(traing_clf.coef_)
                self.binary_clf.append(traing_clf)
        self.classes.remove('RARE_VALUE')
        return self
        
    def predict(self, data_to_predict):
        author_proba = pd.DataFrame()
        for author, author_clf in zip(self.classes, self.binary_clf):
            # only look at probability of 0 (belone to that author)
            if self.clf_type in ["Logistic", "prob_svm"]:
                author_proba[author] = author_clf.predict_proba(data_to_predict)[:,0]
            # distance for svm
            elif self.clf_type == "SVM":
                print(author_clf.decision_function(data_to_predict).shape)
                author_proba[author] = author_clf.decision_function(data_to_predict)
        # for author less than threshold number of samples
        self.predict_proba = author_proba
        print(self.predict_proba)
        if self.clf_type in ["Logistic", "prob_svm"]:
            labels = author_proba.idxmax(axis=1).values
        elif self.clf_type == "SVM":
            labels = author_proba.idxmin(axis=1).values
        return labels
    
    

In [12]:
# load the file
import sys
import io
import os
import collections
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# fix random seed for reproducibility
np.random.seed(1)

fileDir = "../Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(fileDir)

# different embeddings
embeddings = ["pv_dm", "pv_dbow"]
for select_emb in embeddings:
    print("Load embedding: ", select_emb)
    # read pretrained embeddings
    pretrained_emb, pretrained_emb_pid = read_textual_embedding(emb_type=select_emb)
    # collect statistic to output
    allname, positive_sample_size, negative_sample_size = ([] for i in range(3))

    all_LR_accuracy, all_LR_f1, all_svcLinear_accuracy, all_svcLinear_f1 = ([] for i in range(4))

    # read all file in labeled group
    for file in listfiles:
        # group name
        temp = file.split("_")
        name = temp[1]+"_"+temp[-1]
        print("For name: ",name)
        # read needed content in labeled file
        labeled_data = read_labeled_file(fileDir+file)
        # collect all labeled sample
        all_labeled_sample = labeled_data["paperID"].tolist()
        print("Total sample size before apply threshold: ",len(labeled_data))
        # count number of paper each author write based on author ID
        paperCounter = collections.Counter(labeled_data["authorID"])
        print(paperCounter)
        print("Total author before apply threshoid: ", len(paperCounter))
        # collect per class statistic
        for k in list(paperCounter):
            if paperCounter[k] < threshold:
                del paperCounter[k]
        temp =list(paperCounter.keys())
        print(temp)
        # remove authors that write smaller than threshold number of authors
        temp = labeled_data[labeled_data.authorID.isin(temp)]
        author_list = set(temp["authorID"])
        print("Total author after apply threshoid: ", len(author_list))
        # generate label for self defined one vs rest
        labeled_data.loc[labeled_data["authorID"].value_counts()[labeled_data["authorID"]].values < 10, "authorID"] = "RARE_VALUE"
        # if only have one class or no class pass the threshold, not applicable
        if(len(paperCounter)==0) or (len(paperCounter)==1):
            print(name," pass")
        else:
            allname.append(name)
            # shuffle the data
            labeled_data = labeled_data.sample(frac=1).reset_index(drop=True)
            # for each name group
            # read in data
            # list of different data field
            part_collection = []
            # select feature wanted to fit to clustering/classification algorithm
            data_textual = extract_embedding(pretrained_emb, pretrained_emb_pid, labeled_data["paperID"])
            print(data_textual.shape)
            part_collection.append(data_textual)
            # merge different part of data data together by concatenate it all together
            # remove empty emb (when emb set off)
            part_collection = [part for part in part_collection if len(part)!=0]
            print(len(part_collection))
            if len(part_collection)>1:
                combinedata = np.concatenate(part_collection,axis=1)
            elif len(part_collection)==1:
                if isinstance(part_collection[0], pd.DataFrame):
                    combinedata = part_collection[0].values
                else:
                    combinedata = part_collection[0]
            else:
                print("No data available")
                break
            print(combinedata.shape)
            # train test split 1:9 ratio
            X_train, X_test, y_train, y_test = train_test_split(combinedata, labeled_data["authorID"], 
                                                                test_size=0.1, stratify = labeled_data["authorID"])
            # get index of y_train
            train_index = y_train.index.tolist()
            # use it to get label and pid 
            train_label_data = labeled_data.loc[train_index]
            # use logistic regression
            lr_clf = LogisticRegression()
            lr_clf_ovr = one_vs_rest().fit_one_vs_rest(X_train, train_label_data, lr_clf, clf_type = "Logistic")
            lr_label_predict = lr_clf_ovr.predict(X_test)
            print(lr_label_predict)
            print(y_test)
            lr_acc = accuracy_score(y_test, lr_label_predict)
            lr_f1 = f1_score(y_test, lr_label_predict, average='macro')
            print(metrics.classification_report(y_test, lr_label_predict))
            print("LR Accuracy: ",lr_acc)
            print("LR F1: ", lr_f1)
            
            # use svm
            scv_clf = SVC(kernel='linear', probability = True)
            scv_clf_ovr = one_vs_rest().fit_one_vs_rest(X_train, train_label_data, scv_clf, clf_type = "prob_svm")
            svc_label_predict = scv_clf_ovr.predict(X_test)
            svc_acc = accuracy_score(y_test, svc_label_predict)
            svc_f1 = f1_score(y_test, svc_label_predict, average='macro')
            print(svc_label_predict)
            print(y_test)
            print(metrics.classification_report(y_test, svc_label_predict))
            print("SVC Accuracy: ",svc_acc)
            print("SVC F1: ", svc_f1)
            
        break
    break
#     # write evaluation result to excel
#     output = pd.DataFrame({'Author Name':allname,
#                            "positive sample size":positive_sample_size,"negative sample size":negative_sample_size, 
#                            "svc(linear) accuracy":all_svcLinear_accuracy, "svc(linear) f1": all_svcLinear_f1, 
#                            "logistic regression accuracy":all_LR_accuracy, "logistic regression f1": all_LR_f1})

#     savePath = "../result/"+Dataset+"/binary_global_emb_sample=140k/"
#     if not os.path.exists(savePath):
#         os.makedirs(savePath)
#     filename = "textual="+select_emb+"_threshold="+str(threshold)+".csv"
#     output.to_csv(savePath+filename, encoding='utf-8',index=False)
#     print(select_emb, " Done")

Load embedding:  pv_dm
Total textual vector records: 135796
Vector dimension:  100
For name:  j_read
Total sample size before apply threshold:  136
Counter({'0000-0002-5159-1192': 57, '0000-0002-9029-5185': 39, '0000-0002-9697-0962': 31, '0000-0002-4739-9245': 3, '0000-0003-0605-5259': 3, '0000-0003-4316-7006': 1, '0000-0002-0784-0091': 1, '0000-0002-3888-6631': 1})
Total author before apply threshoid:  8
['0000-0002-9697-0962', '0000-0002-9029-5185', '0000-0002-5159-1192']
Total author after apply threshoid:  3
Total missing sample:  0
(136, 100)
1
(136, 100)
Binary clf:  0000-0002-5159-1192
[[ 0.02222388  0.17957971  0.12327487  0.23665527  0.46883194 -0.46871769
  -0.35149199  0.08322612 -0.41095083  0.11814027 -0.44210205  0.46083861
  -0.02273823  0.23913143  0.18668481 -0.34983477 -0.03687433  0.25842248
  -0.21655542 -0.16086778  0.25058371 -0.22296218 -0.26438318 -0.13153422
  -0.49796063 -0.08962691  0.44790371  0.58694331  0.38215174 -0.05784242
  -0.62307023 -0.61337599  0.2

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precis

In [ ]:
# accuracy
from statistics import mean 
cleaned_svcLinear_accuracy = [x for x in all_svcLinear_accuracy if isinstance(x, float)]
cleaned_lr_accuracy = [x for x in all_LR_accuracy if isinstance(x, float)]
print(len(cleaned_svcLinear_accuracy))
print(len(cleaned_lr_accuracy))
print(mean(cleaned_svcLinear_accuracy))
print(mean(cleaned_lr_accuracy))

In [ ]:
# f1
from statistics import mean 
# remove string from result
cleaned_svcLinear_f1 = [x for x in all_svcLinear_f1 if isinstance(x, float)]
cleaned_lr_f1 = [x for x in all_LR_f1 if isinstance(x, float)]
print(len(cleaned_svcLinear_f1))
print(len(cleaned_lr_f1))
print(mean(cleaned_svcLinear_f1))
print(mean(cleaned_lr_f1))

In [ ]:
print(len(listfiles))

In [ ]:
%reset

In [ ]:
%who